In [ ]:
import requests
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import time

In [ ]:
urlRep = 'https://markets.businessinsider.com/news/aapl-stock?p='
response = requests.get(urlRep)
response.status_code, response.reason
headlines = []
dates = []

In [ ]:
def getHeadlines(response):
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.findAll('div', attrs = {'class': 'latest-news__story'})

    for article in articles:
        headline = article.find('a').text
        date = article.find('time')['datetime']
        headlines.append(headline)
        dates.append(date)

        if len(headlines) % 100 == 0:
            print(len(headlines))
            print(date)
    
    if len(articles) == 0:
        print("0 links added, please check")

In [ ]:
urlBI = 'https://markets.businessinsider.com/news/aapl-stock?p='
#urlBI = 'https://markets.businessinsider.com/news/tsla-stock?p='

#define pages to crawl dependent from test range
pages = 300
for i in range(1,pages):
    response = requests.get(urlBI + str(i))
    if response.status_code != 200:
        print(response.status_code, response.reason)
    getHeadlines(response)

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M%S")
df = pd.DataFrame({'Headline': headlines, 'Date': dates})
from datetime import datetime
import pytz

# Convert string to datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
df.head()

In [ ]:
# Define timezone objects for European Summertime and New York
europe_summertime = pytz.timezone('Europe/Paris')
new_york = pytz.timezone('America/New_York')

# Convert datetime object from European Summertime to New York
df['Date'] = df['Date'].apply(lambda x: europe_summertime.localize(x, is_dst=None).astimezone(new_york).replace(tzinfo=None))

df.to_csv('data/headlines/TSLA/BusinessInsider' + timestr + '.csv')